In [1]:
# import library
import numpy as np
import pandas as pd
from pandas import DataFrame
import random
from collections import defaultdict
from sklearn.utils import shuffle
import random
from random import sample


# Load data
random.seed(10)
#data_path = "C:\Users\lulil\Desktop\fastbook\discrete_ql"

#ori_data_all = pd.read_csv("D:\Liling\OneDrive - University of Pittsburgh\master_thesis\q_learning_discrete_state\data\data_all_vaso_penalty1.csv")
data = pd.read_csv("data\data_cont_vaso_reward.csv")
train_ids = pd.read_csv(r"D:\Liling\OneDrive - University of Pittsburgh\master_thesis\q_learning_discrete_state\data\train_id.csv")
test_ids = pd.read_csv(r"D:\Liling\OneDrive - University of Pittsburgh\master_thesis\q_learning_discrete_state\data\test_id.csv")
clin_p_old = pd.read_csv('data\clin_p_old.csv')
clin_p_new = pd.read_csv('data\clin_p_new.csv')

In [ ]:
clin_p_old = clin_p_old.to_numpy()
clin_p_new = clin_p_new.to_numpy()

In [5]:
data = data.loc[data.last_record==0]

train_df = data[data.hosp_id.isin(train_ids.hosp_id.tolist())]
test_df = data[data.hosp_id.isin(test_ids.hosp_id.tolist())]

In [14]:
# QLearning
class QLearningAgent:
    def __init__(self, state_num, action_num):
        self.actions = list(range(action_num))
        self.learning_rate = 0.1
        self.discount_factor = 0.9
        self.epsilon = 0.1
        self.q_table = np.zeros((state_num, action_num), dtype=np.float32)
        
    def learn(self, state, action, reward, next_state):
        cur_q = self.q_table[state][action]
        # Update
        new_q = reward + self.discount_factor * max(self.q_table[next_state])
        self.q_table[state][action] += self.learning_rate * (new_q - cur_q)
        
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.actions)
        else:
            state_actions = self.q_table[state]
            action = self.arg_max(state_action)
    
    @staticmethod
    def arg_max(state_actions):
        max_index_list = []
        max_value = state_actions[0]
        for index in range(len(state_actions)):
            value = state_actions[index]
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
                
        return random.choice(max_index_list)

In [15]:
# Initialize the QLearning
def compute_policy(train_df,state_num,action_num,max_iters,state,next_state,reward_learn,epsilon,clin_p):
  #df = shuffle(train_df)
  
  agent = QLearningAgent(state_num=750, action_num=25)
  
  max_iters =1
  #val = []
  for iter_id in range(max_iters):
      print("Iter [%d/%d]" % (iter_id + 1, max_iters))
      for row_id in train_df.index:
          state1 = train_df.loc[row_id, state]-1
          next_state1 = train_df.loc[row_id, next_state]-1
          action = train_df.loc[row_id, "action"]-1
          action = round(action)
          reward = train_df.loc[row_id, reward_learn]
          #print(state,action)
          agent.learn(state1, action, reward, next_state1)
  policy_table = np.zeros((state_num, action_num), dtype=np.float32)
  for state_id in range(state_num):
      q_row = agent.q_table[state_id]
      action_max = np.argmax(q_row)
      
      policy_table[state_id] = epsilon / action_num
      policy_table[state_id][action_max] += (1.0 - epsilon)    
  ratio_table = policy_table / (clin_p + epsilon / 25)
  return ratio_table,policy_table

In [16]:
#compute optimal policy
intm3_ratio,intm3_p=compute_policy(train_df,750,25,1,"state","next_state","reward_intm3",0.1,clin_p_old)
intm3_ratio_new,intm3_p_new=compute_policy(train_df,750,25,1,"new_state","new_next_state","reward_intm3",0.1,clin_p_new)

vaso1_ratio,vaso1_p=compute_policy(train_df,750,25,1,"state","next_state","reward_vaso1",0.1,clin_p_old)

ai_ratio,ai_p=compute_policy(train_df,750,25,1,"state","next_state","reward_orig",0.1,clin_p_old)

intm1_ratio,intm1_p=compute_policy(train_df,750,25,1,"state","next_state","reward_intm1",0.1,clin_p_old)
intm2_ratio,intm2_p=compute_policy(train_df,750,25,1,"state","next_state","reward_intm2",0.1,clin_p_old)
vaso2_ratio,vaso2_p=compute_policy(train_df,750,25,1,"state","next_state","reward_vaso2",0.1,clin_p_old)
vaso1_ratio,vaso1_p=compute_policy(train_df,750,25,1,"state","next_state","reward_vaso1",0.1,clin_p_old)
vaso3_ratio,vaso3_p=compute_policy(train_df,750,25,1,"state","next_state","reward_vaso3",0.1,clin_p_old)
intm1_ratio_new,intm1_p_new=compute_policy(train_df,750,25,1,"new_state","new_next_state","reward_intm1",0.1,clin_p_new)
intm2_ratio_new,intm2_p_new=compute_policy(train_df,750,25,1,"new_state","new_next_state","reward_intm2",0.1,clin_p_new)
vaso2_ratio_new,vaso2_p_new=compute_policy(train_df,750,25,1,"new_state","new_next_state","reward_vaso2",0.1,clin_p_new)
vaso1_ratio_new,vaso1_p_new=compute_policy(train_df,750,25,1,"new_state","new_next_state","reward_vaso1",0.1,clin_p_new)
vaso3_ratio_new,vaso3_p_new=compute_policy(train_df,750,25,1,"new_state","new_next_state","reward_vaso3",0.1,clin_p_new)

ai_ratio_new,ai_p_new=compute_policy(train_df,750,25,1,"new_state","new_next_state","reward_orig",0.1,clin_p_new)

epsilon=0.1
Rand_p = np.zeros((750,25),dtype=np.float32)+1/25

Rand_ratio = Rand_p/(clin_p_old + epsilon / 25)
Rand_ratio_new = Rand_p/(clin_p_new + epsilon / 25)
Clin_ratio = np.zeros((750,25),dtype=np.float32)+1

Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]
Iter [1/1]


In [18]:
#compute trajectory-wis WIS
def compute_wis(test_df,nboot,reward_learn,state,ai_ratio):
  unique_id=(test_df.hosp_id).unique().tolist()
  wis_i = []
  #nboot =10
  for i in range(nboot):
    cur_hosp_id = ""
    tot_wr = 0.0
    tot_w = 0.0
    sample_id = sample(unique_id,5000)
    test_df1=test_df[test_df.hosp_id.isin (sample_id)]
    test_df1 = test_df1.reset_index(drop=True)
    #for row_id in range(len(test_df)):
    for row_id in test_df1.index:
        hosp_id = test_df1.loc[row_id, "hosp_id"]
        
        if cur_hosp_id != hosp_id:
            if row_id > 0:
                tot_wr += r_i * w_i
                tot_w += w_i
            
            cur_hosp_id = hosp_id        
            w_i = 1.0
            r_i = 0.0
            
        state1 = test_df1.loc[row_id, state] - 1
        #next_state = test_df1.loc[row_id, "next_state"] - 1
        action = test_df1.loc[row_id, "action"] - 1
        action = round(action)
        reward = test_df1.loc[row_id, reward_learn]
        
        w_i *= ai_ratio[state1][action]
        r_i += reward
    
    wis = tot_wr / tot_w
    wis_i.append(wis)
  return wis_i

In [19]:
# evaluated on unconstrained reward framework
wis_intm3=compute_wis(test_df,100,"reward_orig","state",intm3_ratio)
wis_intm3_new=compute_wis(test_df,100,"reward_orig","new_state",intm3_ratio_new)

pd.DataFrame(wis_intm3).to_csv("data_noshuffle\intm3.csv",index=False)
pd.DataFrame(wis_intm3_new).to_csv("data_noshuffle\intm3_new.csv",index=False)

In [20]:
wis_ai=compute_wis(test_df,100,"reward_orig","state",ai_ratio)
wis_ai_new=compute_wis(test_df,100,"reward_orig","new_state",ai_ratio_new)

wis_clin=compute_wis(test_df,100,"reward_orig","state",Clin_ratio)
wis_clin_new=compute_wis(test_df,100,"reward_orig","new_state",Clin_ratio)
wis_intm1=compute_wis(test_df,100,"reward_orig","state",intm1_ratio)
wis_intm1_new=compute_wis(test_df,100,"reward_orig","new_state",intm1_ratio_new)
wis_intm2=compute_wis(test_df,100,"reward_orig","state",intm2_ratio)
wis_intm2_new=compute_wis(test_df,100,"reward_orig","new_state",intm2_ratio_new)
wis_vaso2=compute_wis(test_df,100,"reward_orig","state",vaso2_ratio)
wis_vaso2_new=compute_wis(test_df,100,"reward_orig","new_state",vaso2_ratio_new)
wis_vaso1=compute_wis(test_df,100,"reward_orig","state",vaso1_ratio)
wis_vaso1_new=compute_wis(test_df,100,"reward_orig","new_state",vaso1_ratio_new)
wis_vaso3=compute_wis(test_df,100,"reward_orig","state",vaso3_ratio)
wis_vaso3_new=compute_wis(test_df,100,"reward_orig","new_state",vaso3_ratio_new)
wis_rand=compute_wis(test_df,100,"reward_orig","state",Rand_ratio)
wis_rand_new=compute_wis(test_df,100,"reward_orig","new_state",Rand_ratio_new)

wis_df = pd.DataFrame(list(zip(wis_rand, wis_clin, wis_ai,wis_intm1,\
                            wis_intm2,wis_intm3,wis_vaso1,wis_vaso2,wis_vaso3,\
                            wis_rand_new,wis_ai_new,wis_clin_new,wis_intm1_new,wis_intm2_new,wis_intm3_new,\
                            wis_vaso1_new,wis_vaso2_new,wis_vaso3_new)),\
                   columns =['Random', 'Clinician',"AI","AI_Intm1","AI_Intm2","AI_Intm3",\
                             "AI_vaso1","AI_vaso2","AI_vaso3",'Random_new',"AI_new","Clin_new","AI_Intm1_new",\
                             "AI_Intm2_new","AI_Intm3_new","AI_vaso1_new","AI_vaso2_new","AI_vaso3_new"]) 
wis_df

,Random,Clinician,AI,AI_Intm1,AI_Intm2,AI_Intm3,AI_vaso1,AI_vaso2,AI_vaso3,Random_new,AI_new,Clin_new,AI_Intm1_new,AI_Intm2_new,AI_Intm3_new,AI_vaso1_new,AI_vaso2_new,AI_vaso3_new
0,5.936301,5.908182,8.238747,14.604954,13.347571,14.598630,5.713716,14.499051,14.615562,-0.208770,14.966088,5.722144,14.934723,14.989151,14.986905,14.910879,14.891605,14.943337
1,6.172104,5.842168,14.836725,14.845351,13.161775,14.591653,14.499064,8.223025,14.688864,-1.354204,14.989297,5.722144,14.937061,14.889905,14.960630,14.925564,14.928424,14.922918
2,13.546352,5.902180,14.923213,14.599998,-4.667465,6.960338,6.562456,14.693474,14.696666,-4.407520,13.961348,5.836167,14.934183,14.887770,14.964694,14.916280,14.922416,14.922715
3,3.737739,5.860172,14.593503,14.593716,-3.317742,14.906843,14.737018,14.506152,14.592143,-1.363838,14.937238,5.746149,14.811687,14.957556,14.892861,14.911118,14.916476,14.863682
4,6.147241,5.848170,14.600327,7.648341,12.798295,14.837753,14.482391,14.487802,6.654498,-1.576066,14.965726,5.842168,14.932949,14.941553,14.931221,14.920753,14.913638,14.883002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6.034568,5.902180,14.591450,14.594299,12.546751,-2.296253,14.499277,14.550262,8.151758,-4.339751,14.912281,5.950190,14.954863,14.961132,14.954774,14.912979,14.964786,14.855414
96,5.332429,5.830166,14.600875,14.594243,12.568153,14.586315,4.907745,14.523932,14.598793,-4.287996,14.964775,5.770154,14.947106,14.943389,14.932495,14.941782,14.847963,14.917244
97,5.328647,5.860172,5.057850,7.598737,13.924017,14.603716,14.500840,14.607062,14.584507,-1.196782,14.882407,5.872174,14.934256,14.988859,14.935530,14.866307,14.874035,14.919520
98,6.127449,5.938188,14.600453,14.592783,12.652650,14.585381,14.497962,14.614566,6.655688,-4.953770,14.960714,5.830166,14.935300,14.986563,14.931861,14.919095,14.947511,14.901524


In [41]:
wis_df.to_csv(r"data_noshuffle\wis_df_lastversion.csv")

In [23]:
#evaluated on reward_intm1 framework
wis_intm3=compute_wis(test_df,100,"reward_intm1","state",intm3_ratio)
wis_intm3_new=compute_wis(test_df,100,"reward_intm1","new_state",intm3_ratio_new)

wis_ai=compute_wis(test_df,100,"reward_intm1","state",ai_ratio)
wis_ai_new=compute_wis(test_df,100,"reward_intm1","new_state",ai_ratio_new)

wis_clin=compute_wis(test_df,100,"reward_intm1","state",Clin_ratio)
wis_clin_new=compute_wis(test_df,100,"reward_intm1","new_state",Clin_ratio)
wis_intm1=compute_wis(test_df,100,"reward_intm1","state",intm1_ratio)
wis_intm1_new=compute_wis(test_df,100,"reward_intm1","new_state",intm1_ratio_new)
wis_intm2=compute_wis(test_df,100,"reward_intm1","state",intm2_ratio)
wis_intm2_new=compute_wis(test_df,100,"reward_intm1","new_state",intm2_ratio_new)
wis_vaso1=compute_wis(test_df,100,"reward_intm1","state",vaso1_ratio)
wis_vaso1_new=compute_wis(test_df,100,"reward_intm1","new_state",vaso1_ratio_new)
wis_vaso2=compute_wis(test_df,100,"reward_intm1","state",vaso2_ratio)
wis_vaso2_new=compute_wis(test_df,100,"reward_intm1","new_state",vaso2_ratio_new)
wis_vaso3=compute_wis(test_df,100,"reward_intm1","state",vaso3_ratio)
wis_vaso3_new=compute_wis(test_df,100,"reward_intm1","new_state",vaso3_ratio_new)
wis_rand=compute_wis(test_df,100,"reward_intm1","state",Rand_ratio)
wis_rand_new=compute_wis(test_df,100,"reward_intm1","new_state",Rand_ratio_new)

wis_df_intm1 = pd.DataFrame(list(zip(wis_rand, wis_clin, wis_ai,wis_intm1,\
                            wis_intm2,wis_intm3,wis_vaso1,wis_vaso2,wis_vaso3,\
                            wis_rand_new,wis_ai_new,wis_clin_new,wis_intm1_new,wis_intm2_new,wis_intm3_new,\
                            wis_vaso1_new,wis_vaso2_new,wis_vaso3_new)),\
                   columns =['Random', 'Clinician',"AI","AI_Intm1","AI_Intm2","AI_Intm3",\
                             "AI_vaso1","AI_vaso2","AI_vaso3",'Random_new',"AI_new","Clin_new","AI_Intm1_new",\
                             "AI_Intm2_new", "AI_Intm3_new","AI_vaso1_new","AI_vaso2_new","AI_vaso3_new"]) 
wis_df_intm1

,Random,Clinician,AI,AI_Intm1,AI_Intm2,AI_Intm3,AI_vaso1,AI_vaso2,AI_vaso3,Random_new,AI_new,Clin_new,AI_Intm1_new,AI_Intm2_new,AI_Intm3_new,AI_vaso1_new,AI_vaso2_new,AI_vaso3_new
0,1.029668,6.432896,13.331467,13.258044,-1.822276,13.179909,6.332919,13.047438,13.258135,-3.861774,15.588995,6.490842,13.796855,15.610427,15.572825,16.135273,15.565673,16.087417
1,7.137599,6.345969,13.177025,13.222478,11.954855,6.165187,13.088267,6.143989,13.181500,-1.033782,14.256800,6.361128,15.570282,15.593626,15.578968,15.562885,15.607688,16.204663
2,7.669689,6.431624,13.143205,13.235205,11.224900,13.138767,13.390248,13.064207,7.655371,-0.794820,15.598060,6.513545,15.573080,15.617397,15.611457,15.571197,15.592048,14.258697
3,7.074572,6.378700,13.122754,13.245089,11.224627,13.159943,13.099693,13.123942,13.188248,-3.842299,13.504875,6.463677,15.593244,15.593903,14.383500,15.547516,15.577077,15.575935
4,6.510819,6.479264,7.093843,13.466387,11.434822,13.223090,13.140147,13.047126,13.368964,-8.549151,16.190360,6.389214,16.120464,15.625872,15.629748,14.224088,14.174770,15.582388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6.527885,6.253699,6.350599,13.150130,-2.093177,13.290540,13.137307,13.079666,13.229372,0.867185,15.619786,6.284516,16.115079,15.593748,15.635274,14.276743,16.096923,16.210512
96,1.572491,6.316927,13.171715,13.179819,-5.330255,13.186319,13.174539,13.133622,13.175548,-1.574785,15.577712,6.335078,13.790922,15.607031,15.573577,15.572193,15.592983,15.562400
97,5.438924,6.402253,6.459419,13.223534,11.155235,13.183838,13.211146,13.176572,13.178216,-1.245191,15.606605,6.602701,15.613644,15.683117,14.309961,15.582312,15.560030,15.573821
98,7.546421,6.552201,13.170218,13.194912,-1.600176,13.140153,6.344105,13.101630,13.150648,6.859823,14.255341,6.519399,15.610979,15.607908,15.618261,15.553848,15.577684,14.309310


In [42]:
wis_df_intm1.to_csv(r"data_noshuffle\wis_df_intm1.csv")

In [25]:
#evaluated on reward_intm2 framework
wis_ai=compute_wis(test_df,100,"reward_intm2","state",ai_ratio)
wis_ai_new=compute_wis(test_df,100,"reward_intm2","new_state",ai_ratio_new)
wis_intm3=compute_wis(test_df,100,"reward_intm2","state",intm3_ratio)
wis_intm3_new=compute_wis(test_df,100,"reward_intm2","new_state",intm3_ratio_new)
wis_clin=compute_wis(test_df,100,"reward_intm2","state",Clin_ratio)
wis_clin_new=compute_wis(test_df,100,"reward_intm2","new_state",Clin_ratio)
wis_intm1=compute_wis(test_df,100,"reward_intm2","state",intm1_ratio)
wis_intm1_new=compute_wis(test_df,100,"reward_intm2","new_state",intm1_ratio_new)
wis_intm2=compute_wis(test_df,100,"reward_intm2","state",intm2_ratio)
wis_intm2_new=compute_wis(test_df,100,"reward_intm2","new_state",intm2_ratio_new)
wis_vaso1=compute_wis(test_df,100,"reward_intm2","state",vaso1_ratio)
wis_vaso1_new=compute_wis(test_df,100,"reward_intm2","new_state",vaso1_ratio_new)
wis_vaso2=compute_wis(test_df,100,"reward_intm2","state",vaso2_ratio)
wis_vaso2_new=compute_wis(test_df,100,"reward_intm2","new_state",vaso2_ratio_new)
wis_vaso3=compute_wis(test_df,100,"reward_intm2","state",vaso3_ratio)
wis_vaso3_new=compute_wis(test_df,100,"reward_intm2","new_state",vaso3_ratio_new)

wis_rand=compute_wis(test_df,100,"reward_intm2","state",Rand_ratio)
wis_rand_new=compute_wis(test_df,100,"reward_intm2","new_state",Rand_ratio_new)

wis_df_intm2 = pd.DataFrame(list(zip(wis_rand, wis_clin, wis_ai,wis_intm1,
                            wis_intm2,wis_intm3,wis_vaso1,wis_vaso2,wis_vaso3,\
                            wis_rand_new,wis_ai_new,wis_clin_new,wis_intm1_new,wis_intm2_new,wis_intm3_new,\
                            wis_vaso1_new,wis_vaso2_new,wis_vaso3_new)),\
                   columns =['Random', 'Clinician',"AI","AI_Intm1","AI_Intm2","AI_Intm3",\
                             "AI_vaso1","AI_vaso2","AI_vaso3",'Random_new',"AI_new","Clin_new","AI_Intm1_new",\
                             "AI_Intm2_new", "AI_Intm3_new","AI_vaso1_new","AI_vaso2_new","AI_vaso3_new"])
wis_df_intm2

,Random,Clinician,AI,AI_Intm1,AI_Intm2,AI_Intm3,AI_vaso1,AI_vaso2,AI_vaso3,Random_new,AI_new,Clin_new,AI_Intm1_new,AI_Intm2_new,AI_Intm3_new,AI_vaso1_new,AI_vaso2_new,AI_vaso3_new
0,6.570512,6.326041,12.905700,7.226010,10.937735,10.415209,6.288411,12.848853,13.126685,-1.790012,15.462734,6.327391,15.415454,15.439003,15.457770,15.412273,15.461321,15.432235
1,7.179432,6.247976,9.757730,13.183927,-5.937141,13.047159,12.859030,5.873924,12.938803,-1.033336,15.429531,6.347814,14.155883,13.775380,15.422847,14.129711,15.421144,13.031578
2,7.358441,6.192924,12.929506,13.111898,11.097771,12.938324,12.934872,12.842747,12.948329,-1.526448,15.420632,6.357125,15.505113,15.439652,15.427533,14.055423,15.398017,15.413051
3,7.231485,6.110224,13.041849,13.138124,10.923051,6.929557,12.979737,12.824699,12.953674,-1.601937,15.419427,6.412795,14.176395,14.290661,15.433355,15.867406,14.122221,14.189903
4,5.922090,6.402149,13.121555,12.949586,11.022671,12.910274,7.022696,13.041057,12.993352,-1.478874,12.906137,6.287125,14.154308,15.451632,15.903282,14.156263,14.199619,15.461735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6.636070,6.320314,12.936173,13.122834,-5.818220,6.735376,12.959499,12.840742,13.049065,-1.100762,15.455382,6.344890,14.153944,15.448282,15.935983,15.876804,12.767451,14.234559
96,1.695600,6.224757,6.619734,8.937783,-1.705083,13.487217,12.847106,12.988882,12.942133,3.742174,12.817440,6.337959,14.258253,15.444359,15.931932,15.422928,13.270079,15.414457
97,7.538551,6.309579,12.935214,12.943751,12.325649,13.052717,12.855698,5.900457,13.015445,0.844410,15.904617,6.375630,15.432839,14.210104,15.415337,15.435955,15.402094,15.411650
98,7.327508,6.404651,12.935141,12.945600,10.932214,12.947557,13.080162,12.890092,12.897995,0.730055,15.421326,6.272586,15.429919,15.454800,15.432554,15.895435,15.433789,15.931116


In [43]:
wis_df_intm2.to_csv(r"data_noshuffle\wis_df_intm2.csv")

In [27]:
#evaluated on reward_intm3 framework
wis_ai=compute_wis(test_df,100,"reward_intm3","state",ai_ratio)
wis_ai_new=compute_wis(test_df,100,"reward_intm3","new_state",ai_ratio_new)
wis_intm3=compute_wis(test_df,100,"reward_intm3","state",intm3_ratio)
wis_intm3_new=compute_wis(test_df,100,"reward_intm3","new_state",intm3_ratio_new)
wis_clin=compute_wis(test_df,100,"reward_intm3","state",Clin_ratio)
wis_clin_new=compute_wis(test_df,100,"reward_intm3","new_state",Clin_ratio)
wis_intm1=compute_wis(test_df,100,"reward_intm3","state",intm1_ratio)
wis_intm1_new=compute_wis(test_df,100,"reward_intm3","new_state",intm1_ratio_new)
wis_intm2=compute_wis(test_df,100,"reward_intm3","state",intm2_ratio)
wis_intm2_new=compute_wis(test_df,100,"reward_intm3","new_state",intm2_ratio_new)
wis_vaso1=compute_wis(test_df,100,"reward_intm3","state",vaso1_ratio)
wis_vaso1_new=compute_wis(test_df,100,"reward_intm3","new_state",vaso1_ratio_new)
wis_vaso2=compute_wis(test_df,100,"reward_intm3","state",vaso2_ratio)
wis_vaso2_new=compute_wis(test_df,100,"reward_intm3","new_state",vaso2_ratio_new)
wis_vaso3=compute_wis(test_df,100,"reward_intm3","state",vaso3_ratio)
wis_vaso3_new=compute_wis(test_df,100,"reward_intm3","new_state",vaso3_ratio_new)

wis_rand=compute_wis(test_df,100,"reward_intm3","state",Rand_ratio)
wis_rand_new=compute_wis(test_df,100,"reward_intm3","new_state",Rand_ratio_new)

wis_df_intm3 = pd.DataFrame(list(zip(wis_rand, wis_clin, wis_ai,wis_intm1,
                            wis_intm2,wis_intm3,wis_vaso1,wis_vaso2,wis_vaso3,\
                            wis_rand_new,wis_ai_new,wis_clin_new,wis_intm1_new,wis_intm2_new,wis_intm3_new,\
                            wis_vaso1_new,wis_vaso2_new,wis_vaso3_new)),\
                   columns =['Random', 'Clinician',"AI","AI_Intm1","AI_Intm2","AI_Intm3",\
                             "AI_vaso1","AI_vaso2","AI_vaso3",'Random_new',"AI_new","Clin_new","AI_Intm1_new",\
                             "AI_Intm2_new", "AI_Intm3_new","AI_vaso1_new","AI_vaso2_new","AI_vaso3_new"])
wis_df_intm3

,Random,Clinician,AI,AI_Intm1,AI_Intm2,AI_Intm3,AI_vaso1,AI_vaso2,AI_vaso3,Random_new,AI_new,Clin_new,AI_Intm1_new,AI_Intm2_new,AI_Intm3_new,AI_vaso1_new,AI_vaso2_new,AI_vaso3_new
0,11.332616,6.391950,13.093867,13.292080,-3.099659,13.101760,13.018046,13.194975,13.148862,-0.957505,15.524975,6.299250,15.531866,16.082834,14.277426,15.549386,15.508909,15.627174
1,6.014577,6.543071,13.243717,13.109067,13.226416,13.107817,13.055936,13.095899,13.072026,-1.239876,15.523362,6.450930,15.519501,15.550238,14.213937,15.534077,14.187578,15.503856
2,1.397728,6.349194,13.096496,13.157805,11.123735,13.108985,13.136368,13.007120,13.061821,-0.763048,15.524576,6.453774,16.044870,15.584900,15.571408,15.509108,16.135317,12.926431
3,5.464170,6.475114,13.286519,13.066604,11.131696,13.099910,10.750127,13.036386,13.104200,4.302041,15.565337,6.479081,15.582708,15.549593,15.582124,15.526504,15.553860,15.503289
4,6.095391,6.327877,13.081586,13.117247,11.093456,13.399901,12.973497,4.538540,13.049275,-4.133654,15.524362,6.408506,15.530824,16.175295,14.207168,15.525383,15.493283,16.012901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,12.656395,6.355406,12.155598,13.210204,11.425484,13.292920,13.128980,7.579474,13.093168,-1.303716,15.532565,6.379491,15.519292,15.545792,15.543433,16.007126,15.513613,15.525451
96,6.815999,6.350443,13.092987,11.122838,13.083162,13.104924,12.993368,13.208182,13.109536,-7.606919,16.049818,6.521223,15.576944,15.551556,15.534675,14.316282,15.506640,15.511605
97,2.921239,6.465270,13.289649,13.216669,11.946243,13.205752,13.096729,13.100028,13.296771,6.837706,13.742310,6.384855,16.049185,16.151642,14.209003,14.314034,15.549386,15.538901
98,6.636540,6.486721,13.098462,7.613389,11.107016,13.176049,13.008377,12.978903,7.588754,-1.150565,14.218418,6.330778,15.530800,16.075281,15.553839,15.508348,15.502941,15.508004


In [44]:
wis_df_intm3.to_csv(r"data_noshuffle\wis_df_intm3.csv")

In [28]:
#evaluated on reward_vaso1 framework
wis_ai=compute_wis(test_df,100,"reward_vaso1","state",ai_ratio)
wis_ai_new=compute_wis(test_df,100,"reward_vaso1","new_state",ai_ratio_new)
wis_intm3=compute_wis(test_df,100,"reward_vaso1","state",intm3_ratio)
wis_intm3_new=compute_wis(test_df,100,"reward_vaso1","new_state",intm3_ratio_new)
wis_clin=compute_wis(test_df,100,"reward_vaso1","state",Clin_ratio)
wis_clin_new=compute_wis(test_df,100,"reward_vaso1","new_state",Clin_ratio)
wis_intm1=compute_wis(test_df,100,"reward_vaso1","state",intm1_ratio)
wis_intm1_new=compute_wis(test_df,100,"reward_vaso1","new_state",intm1_ratio_new)
wis_intm2=compute_wis(test_df,100,"reward_vaso1","state",intm2_ratio)
wis_intm2_new=compute_wis(test_df,100,"reward_vaso1","new_state",intm2_ratio_new)
wis_vaso1=compute_wis(test_df,100,"reward_vaso1","state",vaso1_ratio)
wis_vaso1_new=compute_wis(test_df,100,"reward_vaso1","new_state",vaso1_ratio_new)
wis_vaso2=compute_wis(test_df,100,"reward_vaso1","state",vaso2_ratio)
wis_vaso2_new=compute_wis(test_df,100,"reward_vaso1","new_state",vaso2_ratio_new)
wis_vaso3=compute_wis(test_df,100,"reward_vaso1","state",vaso3_ratio)
wis_vaso3_new=compute_wis(test_df,100,"reward_vaso1","new_state",vaso3_ratio_new)

wis_rand=compute_wis(test_df,100,"reward_vaso1","state",Rand_ratio)
wis_rand_new=compute_wis(test_df,100,"reward_vaso1","new_state",Rand_ratio_new)

wis_df_vaso1 = pd.DataFrame(list(zip(wis_rand, wis_clin, wis_ai,wis_intm1,\
                            wis_intm2,wis_intm3,wis_vaso1,wis_vaso2,wis_vaso3,\
                            wis_rand_new,wis_ai_new,wis_clin_new,wis_intm1_new,wis_intm2_new,wis_intm3_new,\
                            wis_vaso1_new,wis_vaso2_new,wis_vaso3_new)),\
                   columns =['Random', 'Clinician',"AI","AI_Intm1","AI_Intm2","AI_Intm3",\
                             "AI_vaso1","AI_vaso2","AI_vaso3",'Random_new',"AI_new","Clin_new","AI_Intm1_new",\
                             "AI_Intm2_new","AI_Intm3_new","AI_vaso1_new","AI_vaso2_new","AI_vaso3_new"]) 
wis_df_vaso1

,Random,Clinician,AI,AI_Intm1,AI_Intm2,AI_Intm3,AI_vaso1,AI_vaso2,AI_vaso3,Random_new,AI_new,Clin_new,AI_Intm1_new,AI_Intm2_new,AI_Intm3_new,AI_vaso1_new,AI_vaso2_new,AI_vaso3_new
0,0.995043,8.777957,20.216109,20.278188,18.517963,20.294311,19.964850,20.165176,12.565361,3.401504,19.940356,8.772373,19.811667,19.813342,19.775098,19.779820,19.778720,19.813157
1,9.134123,8.831427,11.542431,20.114556,-5.850054,20.228696,19.952053,20.069901,20.075479,11.404237,19.812910,8.803287,19.766881,19.815863,19.790611,19.834875,19.889768,19.434857
2,9.218285,8.674558,20.065451,20.049753,7.296587,4.159452,19.956599,20.081805,20.051912,2.672917,19.788833,8.878852,19.787871,19.820184,19.378945,19.781426,19.778669,19.812978
3,15.904988,8.747120,20.071029,20.184129,17.807289,20.069852,20.185780,11.451707,20.058116,2.762081,19.768885,8.964220,19.792586,19.804699,19.793026,19.770499,19.911070,19.794097
4,15.953794,8.823376,20.109928,20.053391,19.564866,13.251054,-1.741372,19.958841,12.254978,5.067183,19.791669,8.757938,19.789865,19.813290,19.459109,19.804183,19.952465,19.797336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8.003448,8.787949,20.101619,20.068332,17.863162,11.372640,19.968567,11.253006,20.055059,2.658551,19.410526,8.886365,19.946849,19.840925,19.774520,19.833156,19.764787,19.781724
96,8.750254,8.920509,20.058117,20.064610,17.952501,20.055625,11.743468,20.165918,20.282160,3.128128,19.799932,8.876041,19.429845,19.968417,19.788121,19.780785,19.838616,19.504523
97,8.177677,8.748004,20.065575,12.317660,17.959265,20.095808,20.052745,19.963274,20.112746,3.138713,19.931437,8.902925,19.786305,19.797107,19.800348,19.767988,19.806961,19.436749
98,9.431115,8.697193,11.493521,7.459677,18.001761,13.472765,12.642934,19.952032,20.266261,-1.340992,19.887918,8.850743,19.496379,19.832465,19.815648,19.354039,19.802146,19.483195


In [45]:
wis_df_vaso1.to_csv(r"data_noshuffle\wis_df_vaso1.csv")

In [30]:
#evaluated on reward_vaso2 framework
wis_ai=compute_wis(test_df,100,"reward_vaso2","state",ai_ratio)
wis_ai_new=compute_wis(test_df,100,"reward_vaso2","new_state",ai_ratio_new)
wis_intm3=compute_wis(test_df,100,"reward_vaso2","state",intm3_ratio)
wis_intm3_new=compute_wis(test_df,100,"reward_vaso2","new_state",intm3_ratio_new)
wis_clin=compute_wis(test_df,100,"reward_vaso2","state",Clin_ratio)
wis_clin_new=compute_wis(test_df,100,"reward_vaso2","new_state",Clin_ratio)
wis_intm1=compute_wis(test_df,100,"reward_vaso2","state",intm1_ratio)
wis_intm1_new=compute_wis(test_df,100,"reward_vaso2","new_state",intm1_ratio_new)
wis_intm2=compute_wis(test_df,100,"reward_vaso2","state",intm2_ratio)
wis_intm2_new=compute_wis(test_df,100,"reward_vaso2","new_state",intm2_ratio_new)
wis_vaso1=compute_wis(test_df,100,"reward_vaso2","state",vaso1_ratio)
wis_vaso1_new=compute_wis(test_df,100,"reward_vaso2","new_state",vaso1_ratio_new)
wis_vaso2=compute_wis(test_df,100,"reward_vaso2","state",vaso2_ratio)
wis_vaso2_new=compute_wis(test_df,100,"reward_vaso2","new_state",vaso2_ratio_new)
wis_vaso3=compute_wis(test_df,100,"reward_vaso2","state",vaso3_ratio)
wis_vaso3_new=compute_wis(test_df,100,"reward_vaso2","new_state",vaso3_ratio_new)

wis_rand=compute_wis(test_df,100,"reward_vaso2","state",Rand_ratio)
wis_rand_new=compute_wis(test_df,100,"reward_vaso2","new_state",Rand_ratio_new)

wis_df_vaso2 = pd.DataFrame(list(zip(wis_rand, wis_clin, wis_ai,wis_intm1,\
                            wis_intm2,wis_intm3,wis_vaso1,wis_vaso2,wis_vaso3,\
                            wis_rand_new,wis_ai_new,wis_clin_new,wis_intm1_new,wis_intm2_new,wis_intm3_new,\
                            wis_vaso1_new,wis_vaso2_new,wis_vaso3_new)),\
                   columns =['Random', 'Clinician',"AI","AI_Intm1","AI_Intm2","AI_Intm3",\
                             "AI_vaso1","AI_vaso2","AI_vaso3",'Random_new',"AI_new","Clin_new","AI_Intm1_new",\
                             "AI_Intm2_new","AI_Intm3_new","AI_vaso1_new","AI_vaso2_new","AI_vaso3_new"]) 
wis_df_vaso2

,Random,Clinician,AI,AI_Intm1,AI_Intm2,AI_Intm3,AI_vaso1,AI_vaso2,AI_vaso3,Random_new,AI_new,Clin_new,AI_Intm1_new,AI_Intm2_new,AI_Intm3_new,AI_vaso1_new,AI_vaso2_new,AI_vaso3_new
0,-5.547540,7.924077,20.164220,11.874024,17.982475,20.066868,19.947021,19.943714,20.049849,2.375627,19.800247,7.920730,19.779561,19.808419,19.472362,19.384273,19.086329,19.775630
1,6.509194,7.713142,20.072257,20.100866,17.945542,4.226389,20.040292,20.132401,20.056875,2.557630,19.944966,7.747151,19.796187,19.812921,19.788505,19.893462,19.355385,19.934562
2,17.189552,7.876590,20.043568,20.047110,18.040185,20.109952,20.154089,20.102611,20.048111,1.758344,19.800454,8.039591,19.401595,19.847959,19.781053,19.900862,19.803675,19.784742
3,15.267963,7.842920,20.060631,7.880948,17.790616,20.055601,10.886598,10.525994,11.894485,-1.182476,19.934475,7.842439,19.784519,19.947672,19.790529,19.914961,19.769267,19.778439
4,4.108268,7.751580,20.060631,20.066512,17.789012,20.272250,20.047320,20.070425,20.276100,5.742241,19.788116,7.956679,19.808920,19.812929,19.954836,19.374871,19.800852,19.799674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6.486847,7.892730,20.406878,20.062384,20.263797,11.110178,12.249382,19.956487,12.389977,-1.013528,19.776078,7.793779,19.814600,19.475178,19.781727,19.776103,19.801327,19.945899
96,-7.499375,7.842117,20.182452,20.053915,-2.826483,20.066032,20.168579,12.246718,20.035179,7.955050,19.807439,7.798667,19.812701,19.802674,19.918391,19.742386,19.779286,19.778459
97,6.214344,7.709802,20.059252,20.055138,17.754659,20.066653,20.293556,20.062117,20.051461,15.914714,19.960594,7.823185,19.789235,19.497716,19.342876,19.776671,19.396408,19.774595
98,6.946513,7.898643,20.283820,11.206203,17.755626,20.068379,19.959125,20.167450,20.156882,2.614753,19.793829,7.686769,19.827107,19.806763,19.813162,19.786302,19.784935,19.779275


In [46]:
wis_df_vaso2.to_csv(r"data_noshuffle\wis_df_vaso2.csv")

In [32]:
#evaluated on reward_vaso3 framework
wis_ai=compute_wis(test_df,100,"reward_vaso3","state",ai_ratio)
wis_ai_new=compute_wis(test_df,100,"reward_vaso3","new_state",ai_ratio_new)
wis_intm3=compute_wis(test_df,100,"reward_vaso3","state",intm3_ratio)
wis_intm3_new=compute_wis(test_df,100,"reward_vaso3","new_state",intm3_ratio_new)
wis_clin=compute_wis(test_df,100,"reward_vaso3","state",Clin_ratio)
wis_clin_new=compute_wis(test_df,100,"reward_vaso3","new_state",Clin_ratio)
wis_intm1=compute_wis(test_df,100,"reward_vaso3","state",intm1_ratio)
wis_intm1_new=compute_wis(test_df,100,"reward_vaso3","new_state",intm1_ratio_new)
wis_intm2=compute_wis(test_df,100,"reward_vaso3","state",intm2_ratio)
wis_intm2_new=compute_wis(test_df,100,"reward_vaso3","new_state",intm2_ratio_new)
wis_vaso1=compute_wis(test_df,100,"reward_vaso3","state",vaso1_ratio)
wis_vaso1_new=compute_wis(test_df,100,"reward_vaso3","new_state",vaso1_ratio_new)
wis_vaso2=compute_wis(test_df,100,"reward_vaso3","state",vaso2_ratio)
wis_vaso2_new=compute_wis(test_df,100,"reward_vaso3","new_state",vaso2_ratio_new)
wis_vaso3=compute_wis(test_df,100,"reward_vaso3","state",vaso3_ratio)
wis_vaso3_new=compute_wis(test_df,100,"reward_vaso3","new_state",vaso3_ratio_new)

wis_rand=compute_wis(test_df,100,"reward_vaso3","state",Rand_ratio)
wis_rand_new=compute_wis(test_df,100,"reward_vaso3","new_state",Rand_ratio_new)

wis_df_vaso3 = pd.DataFrame(list(zip(wis_rand, wis_clin, wis_ai,wis_intm1,\
                            wis_intm2,wis_intm3,wis_vaso1,wis_vaso2,wis_vaso3,\
                            wis_rand_new,wis_ai_new,wis_clin_new,wis_intm1_new,wis_intm2_new,wis_intm3_new,\
                            wis_vaso1_new,wis_vaso2_new,wis_vaso3_new)),\
                   columns =['Random', 'Clinician',"AI","AI_Intm1","AI_Intm2","AI_Intm3",\
                             "AI_vaso1","AI_vaso2","AI_vaso3",'Random_new',"AI_new","Clin_new","AI_Intm1_new",\
                             "AI_Intm2_new","AI_Intm3_new","AI_vaso1_new","AI_vaso2_new","AI_vaso3_new"]) 
wis_df_vaso3

,Random,Clinician,AI,AI_Intm1,AI_Intm2,AI_Intm3,AI_vaso1,AI_vaso2,AI_vaso3,Random_new,AI_new,Clin_new,AI_Intm1_new,AI_Intm2_new,AI_Intm3_new,AI_vaso1_new,AI_vaso2_new,AI_vaso3_new
0,1.643824,7.594876,17.328645,17.324983,-4.068277,17.320711,17.225084,17.260086,12.877941,-2.802366,16.569134,7.410232,17.124971,17.445815,17.365724,17.124383,17.364522,17.356594
1,7.637931,7.302565,9.235460,17.333049,15.152494,9.008488,8.645529,17.222810,17.525032,-2.724052,16.234899,7.475257,17.398456,17.420989,17.389571,17.427413,17.106703,17.355040
2,13.872689,7.555774,17.323383,17.541739,15.393122,13.887232,8.752643,17.243144,17.317467,-3.329487,17.459120,7.456471,17.358974,17.205917,17.383295,17.367564,17.367660,17.377791
3,7.569146,7.392010,17.545099,17.435257,15.289962,17.334313,8.590590,17.283241,13.063340,1.159983,17.387495,7.539282,17.359616,17.178872,17.403549,17.376989,17.356257,17.340460
4,6.168726,7.433020,17.594383,9.147973,-2.212124,17.332195,17.529886,17.222915,17.325828,13.997655,17.367754,7.462863,17.411017,17.375850,17.443979,17.350546,17.343243,17.351170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8.769022,7.440757,17.363257,17.480421,15.218782,15.355730,17.219779,17.224637,17.491265,1.087261,17.360547,7.364068,17.065934,17.384833,17.353182,17.347073,17.383337,17.346368
96,8.386781,7.501381,17.322377,17.321398,16.801388,17.321578,17.214071,17.485900,17.317136,0.468140,16.569012,7.394975,17.388874,17.163403,17.364364,17.104624,17.351002,17.354082
97,-2.274969,7.571212,9.548708,17.532493,15.215589,17.367602,9.182046,17.260763,17.370198,9.597684,17.144923,7.458560,17.401383,17.387226,17.359629,17.170765,17.347626,17.350474
98,-3.525505,7.357577,17.321808,17.329334,15.267921,17.328561,17.342748,17.230215,17.324887,0.131882,17.372171,7.408035,17.275017,17.387574,17.358809,17.093935,17.338906,17.384140


In [47]:
wis_df_vaso3.to_csv(r"data_noshuffle\wis_df_vaso3.csv")

In [27]:

ai_p = pd.DataFrame(ai_p)
intm1_p = pd.DataFrame(intm1_p)
intm2_p = pd.DataFrame(intm2_p)
vaso1_p = pd.DataFrame(vaso1_p)
vaso2_p = pd.DataFrame(vaso2_p)
vaso3_p = pd.DataFrame(vaso3_p)

#Rand_p_new = pd.DataFrame(Rand_p_new)
ai_p_new = pd.DataFrame(ai_p_new)
intm1_p_new = pd.DataFrame(intm1_p_new)
intm2_p_new = pd.DataFrame(intm2_p_new)
vaso1_p_new = pd.DataFrame(vaso1_p_new)
vaso2_p_new = pd.DataFrame(vaso2_p_new)
vaso3_p_new = pd.DataFrame(vaso3_p_new)


ai_p.to_csv(r"data_noshuffle\ai_p.csv", index=False)
#Rand_p.to_csv(r"data_noshuffle\Rand_p.csv", index=False)
intm1_p.to_csv(r"data_noshuffle\intm1_p.csv", index=False)
intm2_p.to_csv(r"data_noshuffle\intm2_p.csv", index=False)
vaso1_p.to_csv(r"data_noshuffle\vaso1_p.csv", index=False)
vaso2_p.to_csv(r"data_noshuffle\vaso2_p.csv", index=False)
vaso3_p.to_csv(r"data_noshuffle\vaso3_p.csv", index=False)
#Rand_p.to_csv(r"data_noshuffle\rand_p.csv", index=False)
#Rand_p_new.to_csv(r"data_noshuffle\rand_p_new.csv", index=False)
ai_p_new.to_csv(r"data_noshuffle\ai_p_new.csv", index=False)
intm1_p_new.to_csv(r"data_noshuffle\intm1_p_new.csv", index=False)
intm2_p_new.to_csv(r"data_noshuffle\intm2_p_new.csv", index=False)
vaso1_p_new.to_csv(r"data_noshuffle\vaso1_p_new.csv", index=False)
vaso2_p_new.to_csv(r"data_noshuffle\vaso2_p_new.csv", index=False)
vaso3_p_new.to_csv(r"data_noshuffle\vaso3_p_new.csv", index=False)

intm3_p = pd.DataFrame(intm3_p)
intm3_p_new = pd.DataFrame(intm3_p_new)
intm3_p.to_csv(r"data_noshuffle\intm3_p.csv", index=False)
intm3_p_new.to_csv(r"data_noshuffle\intm3_p_new.csv", index=False)